In [2]:
in_path =r"D:/rawdata/"   # use your path
out_path= "D:/cleandata/"
import numpy
import pandas as pd
pd.set_option('display.max_columns', 500)
import glob
import re
from nltk import stem
porter = stem.porter.PorterStemmer()
#from gensim import corpora, models, similarities
#import matplotlib.pyplot as plt
#%matplotlib inlineT

In [3]:
allFiles = glob.glob(in_path + "/*.txt")  # just to merge the data
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df_ = pd.read_table(file_,index_col=False, header=0)
    list_.append(df_)
    df = pd.concat(list_)
#df=df[~df.AB.isnull()]
#df=df[(df.DT!='Correction') & (df.DT!='Editorial Material')]
#df.DE.fillna(' ',inplace=True)
#df=df[df.PY<2016]
df=df[df.PY>=2000]
#df.reset_index(inplace=True, drop=True)
#df['id']=df.index
df=df.dropna(subset=["UT"])  # dropping data without a WUS number
df = df.reset_index(drop=True)
df.to_csv(out_path+"articles.csv")
#print len(df)
#node=df[['id','TI','DE','PY','SO','TC']]
#node.rename(columns={'TI':'label'},inplace=True)
#node.to_csv('../csv/nodes.csv',index=False)

In [4]:
for i in range (0, len(df.index)):  #lower-casing all references
    df.loc[i,"CR"]=str(df.loc[i,"CR"]).lower()
    df.loc[i,"SO"]=str(df.loc[i,"SO"]).lower()
    #df.loc[i,"PY"]=int(df.loc[i,"PY"]) # changing published year format to INT
print (df['SO'].count())   
df['SO'] = df['SO'].map({'mis quarterly': 'MISQ', 'information systems research': 'ISR', 'journal of management information systems':'JMIS',
                        'journal of information technology':'JIT', "information systems journal":"ISJ",
                        'journal of strategic information systems':'JSIS','european journal of information systems':'EJIS',
                        'journal of the association for information systems':'JAIS'})
df["CR"].fillna("NaN", inplace=True) # to avoid float error on ref_count string
#df=df.dropna(subset=["SO"])
print (df['SO'].count())

4899
4899


In [5]:
def ref_count(str):
    #counting total references
    ref= [x.strip() for x in str.split(';')] 
    total_reference= len(ref)
    #counting references to the 8
    counts = {"eur j inform syst": 0, "european journal of information systems": 0,
              "mis q":0, "mis quart":0, "mis quarterly" : 0, "management information systems quarterly":0,
              "j manage inform syst":0,"journal of management information systems":0,
              "inform syst res":0,
              "j inf technol":0, "j inform technol":0, "journal of information technology":0,
              "j assoc inf syst":0,
              "inform syst j":0,
              "j strategic inf syst":0, "journal of strategic information systems":0}
    words = [x.strip() for x in str.split(',')]
    for word in words:
        if word in counts:
            counts[word] += 1
    return (counts, total_reference)

In [19]:
def merge_journals(dict): # merging different abrevations of same journal
    dict["MISQ"] = dict["mis q"]+dict["mis quart"] + dict["mis quarterly"] + dict["management information systems quarterly"]
    dict["JIT"]= dict["j inf technol"] + dict ["j inform technol"] + dict["journal of information technology"]
    dict["EJIS"]= dict["eur j inform syst"] + dict["european journal of information systems"]
    dict["JMIS"]= dict["j manage inform syst"] + dict["journal of management information systems"]
    dict["ISR"] = dict["inform syst res"]
    dict["JAIS"]=dict["j assoc inf syst"]
    dict["ISJ"]= dict["inform syst j"]
    dict["JSIS"]=dict["j strategic inf syst"] + dict["journal of strategic information systems"]
                       
    #del dict["MIS QUART"]
    #del dict["J INFORM TECHNOL"]
    return dict

In [34]:
dic=[]  # id, source journal, count of references to each of 8 journals, total references of the articles, year of publication
for i in range (0, len(df.index)):
    b = ref_count(df.loc[i,"CR"])
    a = merge_journals(b[0])
    g= {'id': i,'source': df.loc[i,"SO"],'target':[{'j':'MISQ','count':a["MISQ"]},{'j':'JIT','count':a["JIT"]},
                                              {'j':'EJIS','count':a["EJIS"]},{'j':'JMIS','count':a["JMIS"]},
                                              {'j':'ISR','count':a["ISR"]},{'j':'JAIS','count':a["JAIS"]},
                                              {'j':'ISJ','count':a["ISJ"]},{'j':'JSIS','count':a["JSIS"]}],'total_ref': b[1], 'year': df.loc[i,"PY"] }
    dic.append(g)

In [36]:
total=0 #total number of references of all articles
for article in dic:
    total +=article['total_ref']
total

305881

In [37]:
listofso=['MISQ', 'ISR', 'JMIS','JIT','ISJ', "JSIS", 'EJIS', 'JAIS' ]
total_ref_per_source={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0} # total references of each journal
for article in dic:
    source=article['source']
    refcount=article['total_ref']
    for so in listofso:
        if source==so:
            total_ref_per_source[so]=total_ref_per_source[so]+refcount

In [47]:
total_ref_per_source

{'EJIS': 44832,
 'ISJ': 25727,
 'ISR': 43009,
 'JAIS': 31803,
 'JIT': 27262,
 'JMIS': 48097,
 'JSIS': 23925,
 'MISQ': 61226}

In [12]:
listofj=['MISQ', 'ISR', 'JMIS','JIT','ISJ', "JSIS", 'EJIS', 'JAIS' ]
listofso=['MISQ', 'ISR', 'JMIS','JIT','ISJ', "JSIS", 'EJIS', 'JAIS' ]
final={'ISJ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JSIS':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'ISR':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'MISQ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JAIS' :{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "EJIS":{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JIT':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "JMIS": {'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0}
      }
for article in dic:
    source=article['source']
    targets=article['target']
    for target in targets:
        jname=target['j']
        jcount=target['count']
        for so in listofso:
            for ta in listofj:
                if source==so:
                    if jname==ta:
                        final[ta][so]=final[ta][so]+jcount
                

In [16]:
Source_Sum={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0} # number of total paper from each journal
k=0
for article in dic:
    for so in listofso:
        if so==article['source']:
            k+=1
            Source_Sum[so]=Source_Sum[so]+1
print (Source_Sum,k) 

{'JAIS': 408, 'MISQ': 803, 'EJIS': 775, 'ISR': 709, 'JSIS': 394, 'JIT': 517, 'ISJ': 467, 'JMIS': 826} 4899


In [19]:
Source_Sum1={}
for so in listofso:
    Source_Sum1[so] = Source_Sum[so]*100/k
print(Source_Sum1)

{'ISR': 14.301225376143943, 'JAIS': 6.32852489530014, 'ISJ': 10.128742050566155, 'EJIS': 15.076779897626803, 'JMIS': 13.15340468434931, 'JIT': 11.292073832790445, 'MISQ': 21.824104234527688, 'JSIS': 7.895145028695517}


In [10]:
citation=pd.DataFrame(data=final)
given_citation = citation.sum(axis=0) # number of times each journal cited any of 8
recieved_citation = citation.sum(axis=1) # number of citation each journal recieved from other 8
print(given_citation, recieved_citation)

EJIS      8729
ISJ_S        0
ISR       6300
JAIS      6164
JIT       4326
JMIS      9202
JSIS      5061
MISQ     11232
dtype: int64 EJIS     3731
ISJ      1500
ISR     10844
JAIS     2406
JIT      1858
JMIS     8166
JSIS     2447
MISQ    20062
dtype: int64


In [15]:
print (recieved_citation.sum(), given_citation.sum())

51014 51014


In [142]:
citation_per = citation.copy()

In [155]:
for so in listofso:
    for ta in listofj:
        citation_per.loc[ta,so]= (citation.loc[ta,so]/given_citation[so]*100)
print (citation_per)

          EJIS       ISJ       ISR      JAIS       JIT      JMIS      JSIS  \
EJIS  20.15004  8.659595  2.831114  5.207657  9.932968  2.161523  6.327498   
ISJ   4.490702  19.22854  1.457603  3.520441  5.748527  1.182923  2.881109   
ISR   15.33178  14.38766  35.33287   20.8793  12.97989   19.8731  12.91028   
JAIS  4.025782  3.490839  3.069376  10.48021  4.834451  2.494892  2.607586   
JIT   4.459003  5.515911  1.177295  2.839066  15.11274  1.247446   5.39752   
JMIS  10.88335  8.640309  12.62789  12.54056  9.343896  38.79987  9.865062   
JSIS  3.476331  4.300868  1.639804  1.995457  5.748527  1.462523   24.9453   
MISQ  37.18301  35.77628  41.86405  42.53731    36.299  32.77772  35.06565   

          MISQ  
EJIS  3.767649  
ISJ    1.79558  
ISR   22.29896  
JAIS   4.19736  
JIT   1.557704  
JMIS  10.93462  
JSIS  1.956722  
MISQ  53.49141  


In [156]:
citation_per1 = citation_per.applymap( lambda x : str(x + "%"))

TypeError: ("unsupported operand type(s) for +: 'numpy.float64' and 'str'", 'occurred at index EJIS')

In [456]:
#chi hast table va koja be dast oomade o che natije dade be ma

In [183]:
given_citation["ISJ"]

5185.0

In [216]:
for i
df1=df[df.PY>=2000]
df1 = df1.reset_index(drop=True)

In [194]:
year=[]  # citation recieved and gave per year
for j in range (2000,2019):
    df1 = df[df.PY==j]
    df1 = df1.reset_index(drop=True)
    dic_new=[]
    for i in range (0, len(df1.index)):
        b = ref_count(df1.loc[i,"CR"])
        a = merge_journals(b)
        g= {'id': i,'source': df1.loc[i,"SO"],'target':[{'j':'MISQ','count':a["MISQ"]},{'j':'JIT','count':a["JIT"]},
                                              {'j':'EJIS','count':a["EJIS"]},{'j':'JMIS','count':a["JMIS"]},
                                              {'j':'ISR','count':a["ISR"]},{'j':'JAIS','count':a["JAIS"]},
                                              {'j':'ISJ','count':a["ISJ"]},{'j':'JSIS','count':a["JSIS"]}]}
        dic_new.append(g)
    final_new={'ISJ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JSIS':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'ISR':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'MISQ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JAIS' :{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "EJIS":{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JIT':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "JMIS": {'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0}
                }
    for article in dic_new:
        source=article['source']
        targets=article['target']
        for target in targets:
            jname=target['j']
            jcount=target['count']
            for so in listofso:
                for ta in listofj:
                    if source==so:
                        if jname==ta:
                            final_new[so][ta]=final_new[so][ta]+jcount
    citation_new=pd.DataFrame(data=final_new)
    given_citation_new = citation_new.sum(axis=0) # number of times each journal cited any of 8
    recieved_citation_new= citation_new.sum(axis=1) # number of citation each journal recieved from other 8
    year.append((j,given_citation_new,recieved_citation_new))

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [262]:
k=2 # 2 for recieved citation, 1 for given citation
M=18
EJIS_R=np.arange(M)
ISJ_R=np.arange(M)
ISR_R=np.arange(M)
JAIS_R=np.arange(M)
JIT_R=np.arange(M)
JMIS_R=np.arange(M)
JSIS_R=np.arange(M)
MISQ_R=np.arange(M)
for i in range (0,18):
    EJIS_R[i]=year[i][k][0]
    ISJ_R[i]=year[i][k][1]
    ISR_R[i]=year[i][k][2]
    JAIS_R[i]=year[i][k][3]
    JIT_R[i]=year[i][k][4]
    JMIS_R[i]=year[i][k][5]
    JSIS_R[i]=year[i][k][6]
    MISQ_R[i]=year[i][k][7]

In [263]:
EJIS_R

array([ 51,  51,  80,  73,  79, 146, 165, 213, 218, 226, 270, 258, 365,
       362, 370, 372, 330, 413])

In [260]:
qqq = np.arange(N)
qqq[0]=2
qqq[0]

2

In [210]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
N = 18
ind = np.arange(N)  # the x locations for the groups
width = 0.15       # the width of the bars

fig = plt.figure()
ax = fig.add_subplot(111)
'EJIS','ISJ','ISR', 'JAIS', 'JIT', 'JMIS','JSIS','MISQ'

yvals = EJIS_R
rects1 = ax.bar(ind, yvals, width, color='b')
zvals = ISJ_R
rects2 = ax.bar(ind+width, zvals, width, color='g')
kvals = ISR_R
rects3 = ax.bar(ind+width*2, kvals, width, color='r')
avals = JAIS_R
rects4 = ax.bar(ind+width*3, avals, width, color='c')
bvals = JIT_R
rects5 = ax.bar(ind+width*4, bvals, width, color='m')
cvals = JMIS_R
rects6 = ax.bar(ind+width*5, cvals, width, color='y')
dvals = JSIS_R
rects7 = ax.bar(ind+width*6, dvals, width, color='k')
evals = MISQ_R
rects8 = ax.bar(ind+width*7, evals, width, color='w')

ax.set_ylabel('Scores')
ax.set_xticks(ind+width)
ax.set_xticklabels( ('2000', '2001', '2002','2003',"2004","2005","2006","2007","2008",'2009', '2010', '2011','2012', "2013","2014","2015","2016"
                     ,"2017","2018" ) )
ax.legend( (rects1[0], rects2[0], rects3[0], rects4[0], rects5[0], rects6[0], rects7[0], rects8[0]), ('EJIS','ISJ','ISR', 'JAIS', 'JIT', 'JMIS',
                                                                                                      'JSIS','MISQ') )

def autolabel(rects):
    for rect in rects:
        h = rect.get_height()
        ax.text(rect.get_x()+rect.get_width()/2., 1*h,
                ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
autolabel(rects4)
autolabel(rects5)
autolabel(rects6)
autolabel(rects7)
autolabel(rects8)

plt.show()

In [219]:
dic_2000=[]
for i in range (0, len(df1.index)):
    b = ref_count(df1.loc[i,"CR"])
    a = merge_journals(b)
    g= {'id': i,'source': df1.loc[i,"SO"],'target':[{'j':'MISQ','count':a["MISQ"]},{'j':'JIT','count':a["JIT"]},
                                              {'j':'EJIS','count':a["EJIS"]},{'j':'JMIS','count':a["JMIS"]},
                                              {'j':'ISR','count':a["ISR"]},{'j':'JAIS','count':a["JAIS"]},
                                              {'j':'ISJ','count':a["ISJ"]},{'j':'JSIS','count':a["JSIS"]}]}
    dic_2000.append(g)

In [220]:
final_2000={'ISJ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JSIS':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'ISR':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'MISQ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JAIS' :{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "EJIS":{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JIT':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "JMIS": {'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0}
      }
for article in dic_2000:
    source=article['source']
    targets=article['target']
    for target in targets:
        jname=target['j']
        jcount=target['count']
        for so in listofso:
            for ta in listofj:
                if source==so:
                    if jname==ta:
                        final_2000[so][ta]=final_2000[so][ta]+jcount

In [222]:
citation_2000=pd.DataFrame(data=final_2000)
given_citation_2000 = citation_2000.sum(axis=0) # number of times each journal cited any of 8
recieved_citatio_2000 = citation_2000.sum(axis=1) # number of citation each journal recieved from other 8

In [234]:
citation_per_2000= citation_2000
for so in listofso:
    for ta in listofj:
        citation_per_2000.loc[ta,so]= citation_2000.loc[ta,so]/given_citation_2000[so]

In [233]:
output= pd.Series([round(val, 2) for val in citation_per_2000])
df['var3'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df['var3']], index = df.index)

,EJIS,ISJ,ISR,JAIS,JIT,JMIS,JSIS,MISQ
EJIS,0.205866%,0.085911%,0.029365%,0.052077%,0.100786%,0.021626%,0.061648%,0.042824%
ISJ,0.047886%,0.202759%,0.015873%,0.035204%,0.063338%,0.011954%,0.030231%,0.020299%
ISR,0.156719%,0.146321%,0.370159%,0.208793%,0.139852%,0.198978%,0.131199%,0.245459%
JAIS,0.043648%,0.037834%,0.034762%,0.104802%,0.055016%,0.025212%,0.028255%,0.048700%
JIT,0.042044%,0.053930%,0.012698%,0.028391%,0.151410%,0.012606%,0.052756%,0.017628%
JMIS,0.106198%,0.087166%,0.121270%,0.125406%,0.092233%,0.387850%,0.096028%,0.111111%
JSIS,0.033223%,0.043687%,0.017778%,0.019955%,0.056403%,0.014779%,0.257064%,0.021457%
MISQ,0.364417%,0.342391%,0.398095%,0.425373%,0.340962%,0.326994%,0.342818%,0.492521%


In [236]:
citation_per_2000

,EJIS,ISJ,ISR,JAIS,JIT,JMIS,JSIS,MISQ
EJIS,0.000024%,0.000018%,0.000005%,0.000008%,0.000023%,0.000002%,0.000012%,0.000004%
ISJ,0.000005%,0.000042%,0.000003%,0.000006%,0.000015%,0.000001%,0.000006%,0.000002%
ISR,0.000018%,0.000031%,0.000059%,0.000034%,0.000032%,0.000022%,0.000026%,0.000022%
JAIS,0.000005%,0.000008%,0.000006%,0.000017%,0.000013%,0.000003%,0.000006%,0.000004%
JIT,0.000005%,0.000011%,0.000002%,0.000005%,0.000035%,0.000001%,0.000010%,0.000002%
JMIS,0.000012%,0.000018%,0.000019%,0.000020%,0.000021%,0.000042%,0.000019%,0.000010%
JSIS,0.000004%,0.000009%,0.000003%,0.000003%,0.000013%,0.000002%,0.000051%,0.000002%
MISQ,0.000042%,0.000072%,0.000063%,0.000069%,0.000079%,0.000036%,0.000068%,0.000044%


In [20]:
#breakdown of papers by years